In [2]:
import json

# Let's load Wilhelm's Translation of the I-Ching, and let's poor it into a JSON & CSV file

In `./iching_fulltext.md` we have a full english translation of the Wilhelm I-Ching translation. We want to write a simple parser which will take out all required pieces from the text and store it inside ann appropriately constructed JSON/CSV.

In [20]:
count = 0
with open('./data/iching_willhelm_markdown.md') as f:
    lines = f.readlines()

with open('./data/i-ching-basic.js') as json_file:
    iching_basic = json.load(json_file)

iching_full_text = ''.join(lines)

def iching_raw_text_section_split():
    """ Split markdown into \n-splitted sections, each on a hexagram. Take section 10-74 of the markdown for this as Hexagram Texts """
    start_ii = 10 # hex 1
    end_ii = 74 # hex 64

    # dictionary prep
    iching_raw = {}
    iching_raw['hexagrams'] = {}

    # Split translation by subsectoin (## )
    split = iching_full_text.split("## ")

    # Now join sections back together, but split them on new lines
    for i in range(start_ii, end_ii):
        iching_raw['hexagrams'][i-start_ii] = [p.split("\n") for p in split][i]

    return iching_raw

iching_raw = iching_raw_text_section_split()

In [21]:
# preview
iching_raw['hexagrams'][55][:20]

['56. Lü / The Wanderer',
 '[Back to Index](#index)',
 '',
 'above LI THE CLINGING, FIRE',
 '',
 'below KêN KEEPING STILL, MOUNTAIN',
 '',
 "The mountain, Kên, stands still; above it fire, Li, flames up and does not tarry. Therefore the two trigrams do not stay together. Strange lands and separation are the wanderer's lot.",
 '',
 'THE JUDGMENT',
 '',
 'The Wanderer. Success through smallness.',
 'Perseverance brings good fortune to the wanderer.',
 '',
 'WHEN A man is a wanderer and stranger, he should not be gruff nor overbearing. He has no large circle of acquaintances, therefore he should not give himself airs. He must be cautious and reserved; in this way he protects himself from evil. If he is obliging toward others, he wins success.',
 '',
 'A wanderer has no fixed abode; his home is the road. Therefore he must take care to remain upright and steadfast, so that he sojourns only in the proper places, associating only with good people. Then he has good fortune and can go his way u

In [22]:
keywords = [ 'above', 'below', 'THE JUDGMENT', 'THE IMAGE', 'THE LINES' ]

We want to do the following:
- identify lines with above keywords
- Extract parts of text into json structure (if applicable)

In [24]:
def wilhelm_hexagram(iching_hexagrams, idx):
    """ Get keywords positions (line number) and hexagram text of hexagram `idx` (eg 63) from iching_hexagrams
        Note that here: Hexagram 1 = iching_hexagrams[0], Hexagram 64 = iching_hexagrams[63]
    """

    # get a single iching hexagram
    hexg = iching_hexagrams[idx]

    # create indice arrray where keywords occur
    idx_array = {}

    # iteratie through lines in hexagram
    for i, l in enumerate(hexg):

        # now we're gonna check if the line contains a special keyword
        for k in keywords:
            
            # if keyword in line
            if k in l[:len(k)]:
                idx_array[k] = i

    return hexg, idx_array

Todo:
- Get started by adding to a dictionary of symbolic representation of the hexagram.
- Judgment section could perhaps be renamed to "Law", and so is everyone writing their own Law, if allowed to have their own interpretation.

In [6]:
# key-map values on hex-value
iching_extented_start = { item['hex']:item for ii, item in enumerate(iching_basic) }

## Let's define core functions

In [7]:
def get_basic_info(hex_text, indices):
    """ Retrieve basic info / name of hexagram, given hex_text and keyword indices"""
    return hex_text[0]

def retrieve_above(hex_text, indices):
    """ Retrieve above, and interpretation, given hex_text and keyword indices"""
    above_text = hex_text[indices['above']]
    return above_text

def retrieve_below(hex_text, indices):
    """ Retrieve below, and interpretation, given hex_text and keyword indices"""
    below_text = hex_text[indices['below']]
    return below_text

def retrieve_symbolic(hex_text, indices):
    """ Retrieve symbolc, and interpretation, given hex_text and keyword indices"""
    # print (indices)
    symbolic_text = '\n'.join( hex_text[indices['below']+1: indices['THE JUDGMENT']])
    return symbolic_text

def retrieve_judgment(hex_text, indices):
    """ Retrieve judgment, and interpretation, given hex_text and keyword indices"""
    judgment_text = '\n'.join( hex_text[indices['THE JUDGMENT']+1: indices['THE IMAGE']])
    return judgment_text

def retrieve_image(hex_text, indices):
    """ Retrieve image, and interpretation, given hex_text and keyword indices"""
    image_text = '\n'.join( hex_text[indices['THE IMAGE']+1: indices['THE LINES']])
    return image_text

def retrieve_lines(hex_text, indices):
    """ Retrieve lines, and interpretation, given hex_text and keyword indices"""
    lines_text = '\n'.join( hex_text[indices['THE LINES']+1:])
    return lines_text

Now go through all hexagrams in our basic-hex kit, and enricht it with Wilhelm

In [9]:
for kk, hexagram in iching_extented_start.items():

    # retrieve hexagram text kk (64 hexagrams total), and markers with core info in text
    hex_wilhelm, markers = wilhelm_hexagram(iching_raw['hexagrams'], kk-1)

    # Enrich hexagram with `hex_wilhelm` info
    hexagram['wilhelm_above'] = retrieve_above(hex_wilhelm, markers)
    hexagram['wilhelm_below'] = retrieve_below(hex_wilhelm, markers)
    hexagram['wilhelm_symbolic'] = retrieve_symbolic(hex_wilhelm, markers)
    hexagram['wilhelm_judgment'] = retrieve_judgment(hex_wilhelm, markers)
    hexagram['wilhelm_image'] = retrieve_image(hex_wilhelm, markers)
    hexagram['wilhelm_lines'] = retrieve_lines(hex_wilhelm, markers)

In [10]:
iching_extented_start[64]

{'hex': 64,
 'hex_font': '䷿',
 'trad_chinese': '未濟',
 'pinyin': 'wèijì',
 'english': 'NotYetFulfilled',
 'binary': 101010,
 'od': 63,
 'wilhelm_above': 'above LI THE CLINGING, FLAME',
 'wilhelm_below': "below K'AN THE ABYSMAL, WATER",
 'wilhelm_symbolic': "\nThis hexagram indicates a time when the transition from disorder to order is not yet completed. The change is indeed prepared for, since all the lines in the upper trigram are in relation to those in the lower. However, they are not yet in their places. While the preceding hexagram offers an analogy to autumn, which forms the transition from summer to winter, this hexagram presents a parallel to spring, which leads out of winter's stagnation into the fruitful time of summer. With this hopeful outlook the Book of Changes come to its close.\n",
 'wilhelm_judgment': '\nBEFORE COMPLETION. Success.\nBut if the little fox, after nearly completing the crossing,\nGets his tail in the water,\n\nThere is nothing that would further.\n\nThe co

## Write to I-Ching Wilhelm Translation

In [12]:
with open('./data/iching_wilhelm_translation.json', 'w') as outfile:
    json.dump(iching_extented_start, outfile)